In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('complete_dataframe.csv')
print(df.head(5))

   Unnamed: 0 author                                               link  \
0           0    mrk  https://ceticismopolitico.com/2017/11/30/katia...   
1           1   None  https://ceticismopolitico.com/2017/11/24/dr-ra...   
2           2   None  https://afolhabrasil.com.br/politica/reinaldo-...   
3           3   None  https://www.diariodobrasil.org/relatorio-assus...   
4           4   None  https://www.diariodobrasil.org/radialista-amer...   

   category date_of_publication  number_of_tokens  \
0  politica          2017-11-30               211   
1  politica          2017-11-24               289   
2  politica          2017-05-23               304   
3  politica          24/07/2017               639   
4  politica          25/07/2017               128   

   number_of_word_without_ponctuation  number_of_types  \
0                                 185              120   
1                                 254              163   
2                                 275              170  

In [42]:
import string 
preprocessed_split=df['preprocessed'].apply(lambda x: x.split())

print(preprocessed_split[0])

#remove pontuação
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in preprocessed_split[0]]

#remove strings vazias
stripped = [string for string in stripped if string != '']
print(stripped)

['katia', 'abreu', 'diz', 'vai', 'colocar', 'expulsao', 'moldura', 'reclamar', 'senadora', 'katia', 'abreu', 'partidoto', 'disse', 'expulsao', 'pmdb', 'resultado', 'acao', 'cupula', 'atual', 'legenda', 'segundo', 'oportunista', '"', 'amanha', 'vou', 'botar', 'moldura', 'dourada', 'expulsao', 'porque', 'maos', 'onde', 'veio', 'atestado', 'boa', 'conduta', 'curriculo', 'pessoas', 'expulsaram', 'servem', 'pais', 'servem', 'pais', 'beneficios', 'proprios', '"', 'disse', 'katia', 'abreu', 'ue', 'expulsao', 'algo', 'tao', 'bom', 'curriculo', 'tanta', 'choradeira', 'katia', 'sabemos', 'motivo', 'provavelmente', 'katia', 'valor', 'pt', 'partido', 'deveria', 'tela', 'absorvido', 'parece', 'pt', 'gostava', 'katia', 'somente', 'ficasse', 'entrincheirada', 'dentro', 'pmdb', 'rebaixar', 'demais', 'resta']
['katia', 'abreu', 'diz', 'vai', 'colocar', 'expulsao', 'moldura', 'reclamar', 'senadora', 'katia', 'abreu', 'partidoto', 'disse', 'expulsao', 'pmdb', 'resultado', 'acao', 'cupula', 'atual', 'lege

In [284]:
from gensim.models import KeyedVectors
# Load vectors directly from the file
# utilizar os modelos provenientes de http://nilc.icmc.usp.br/embeddings
model = KeyedVectors.load_word2vec_format('skip_s300.txt', binary=False)

In [285]:
import numpy as np
word = preprocessed_split[0]
# as palavras que não estão no vocabulários são substituidas por 'unk' = unknown
word = [w if w in model.vocab else 'unk' for w in word]

# tokeniza as palavras 
vectors = [model[w] for w in word if w in model.vocab]
np.array(vectors).shape

(99, 300)

In [286]:
df = pd.read_csv('preprocessed/pre-processed.csv')
print(df.head(5))

preprocessed_split=df['preprocessed_news'].apply(lambda x: x.split())

preprocessed_split2 = preprocessed_split .apply(lambda x: [w if w in model.vocab else 'unk' for w in x])

vector = preprocessed_split2.apply(lambda x:  [model[w] for w in x if w in model.vocab])


   index label                                  preprocessed_news
0      0  fake  katia abreu diz vai colocar expulsao moldura n...
1      1  fake  ray peita bolsonaro conservador fake entrevist...
2      2  fake  reinaldo azevedo desmascarado policia federal ...
3      3  fake  relatorio assustador bndes mostra dinheiro pub...
4      4  fake  radialista americano fala sobre pt vendem ilus...


In [287]:
averaged_vector = vector.apply(lambda x: np.mean(np.array(x),axis=0))

print(averaged_vector[2].shape)

(300,)


In [288]:
X = np.stack( averaged_vector, axis=0 )


In [289]:
y = np.array(df['label'].apply(lambda x:float(x=='true')))

In [303]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=y_test.shape[0], random_state=0)


print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(4320, 300) (4320,)
(1440, 300) (1440,)
(1440, 300) (1440,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

model =  make_pipeline(PolynomialFeatures(2),StandardScaler(),LogisticRegression(penalty='none',max_iter=10000))
model.fit(X_train, y_train)

In [ ]:
print(model.score(X_train,y_train))
print(model.score(X_val,y_val))
print(model.score(X_test,y_test))

In [308]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
# predict probabilities
lr_probs = model.predict_proba(X_test)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# predict class values
yhat = model.predict(X_test)
lr_precision, lr_recall, _ = precision_recall_curve(y_test, lr_probs)
lr_f1, lr_auc = f1_score(y_test, yhat), auc(lr_recall, lr_precision)
# summarize scores
print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))

Logistic: f1=0.866 auc=0.881
